<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun  6 18:05:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os 
os.chdir('/content/')

!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

Cloning into 'TrackNet-Padel'...
remote: Enumerating objects: 890, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (79/79), done.


In [ ]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

### Predict

In [7]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean4'
for i in range(1,2):
  video = 'raw_data/points/clean/PremierRoma_Semis_LGBC_2_25_'+str(i)+'.mp4'
  !python predict.py --video_path={video} --load_weights={weights_path}

2022-06-02 13:55:03.763298: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Number of frames in video and dictionary are not the same!
Fail to load, predict only.
Beginning predicting......
Done......


### Test accuracy

In [12]:
file_name = 'Final_PremierQatar_Tie'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean5'
point_n = 0
for point_n in range(9,13):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} --load_weights={weights_path} # 2>&1 | tee {res}

raw_data/points/Final_PremierQatar_Tie_9.mp4
2022-06-06 17:58:52.777188: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 137
Number of true negative: 63
Number of false positive FP1: 30
Number of false positive FP2: 4
Number of false negative: 154
Accuracy: 0.5154639175257731
Precision: 0.8011695906432749
Recall: 0.47079037800687284
Total Time: 39.283549070358276
(ACC + Pre + Rec)/3: 0.5958079620586404
Done......
raw_data/points/Final_PremierQatar_Tie_10.mp4
2022-06-06 17:59:44.212497: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predictin

### Train Padel

In [ ]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/clean/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

In [ ]:
!rm -rf train_data/*
!python video2img.py 

*   TrackNet-PadelClean2: Entrenado solo con padel, pocos frames (5000) limpios (solo fragmentos con la bola en juego). --lr=10
*   TrackNet-PadelClean3: Continuación, 50 epochs --batch_size=5, con 7k frames (alcanza 0.956 en la validación. - loss:1.20e-4)
*   TrackNet-PadelClean4: Continuación, +50epochs --bacth_size=5, con 12k frames, --lr=1 (alcanza 0.94 en la validación. - loss:1.18e-4)
*   TrackNet-PadelClean5: Continuación, +XXepochs --bacth_size=8, con 20k frames, --lr=1

In [10]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean5'
!python train.py --save_weights={weights_path}  --batch_size=5 --lr=10  --pre_trained=True  --load_weights={weights_path}

lerning rate: 10.0
2022-06-06 07:57:52.059197: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 4090, Learning rate: 1.0000==========
818/818 [==============================] - 595s 708ms/step - loss: 1.5403e-04 - binary_accuracy: 0.9996
==========Epoch 0 start validation==========
Epoch 0 accuracy: 0.779
Epoch 0 precision: 0.962
Epoch 0 recall: 0.778
Epoch 0 average = (accuracy + precision + recall)/3: 0.839
==========Epoch 1, Train steps: 4090, Learning rate: 1.0000==========
818/818 [==============================] - 568s 694ms/step - loss: 1.3029e-04 - binary_accuracy: 0.9996
==========Epoch 1 start validation==========
Epoch 1 accuracy: 0.916
Epoch 1 precision: 0.972
Epoch 1 recall: 0.930
Epoch 1 average = (accuracy + precision + recall)/3: 0.939
==========Epoch 2, Train steps: 4090, Le